In [29]:
import sys
path = '/home/artkaz/arc/arcadia/cloud/analytics/python/lib/clan_tools/src'
if not (path in sys.path):
    sys.path.append(path)
from clan_tools.data.holidays_in_russia import holidays_in_russia_df
from ch_connection import ch_password

In [30]:
holidays = holidays_in_russia_df[holidays_in_russia_df['holiday'] == 1].rename({'date':'ds'}, axis=1)

In [31]:
holidays['holiday'] = 'holiday'

In [32]:
holidays

,ds,holiday
0,1999-01-01,holiday
1,1999-01-02,holiday
2,1999-01-03,holiday
3,1999-01-04,holiday
5,1999-01-06,holiday
...,...,...
9850,2025-12-20,holiday
9851,2025-12-21,holiday
9857,2025-12-27,holiday
9858,2025-12-28,holiday


In [33]:
ch_password

'ZJ5-fk?wsu5eopMq'

In [34]:
import pandas as pd
from fbprophet import Prophet
from io import StringIO
from scipy import stats
import numpy as np
import yt.wrapper


from clan_tools.data_adapters.ClickHouseAdapter import ClickHouseAdapter

from tqdm import tqdm

ch = ClickHouseAdapter( user= 'artkaz',
                        password = ch_password
)

print(ch.execute_query('SELECT now() as now',to_pandas=True))

INFO:clan_tools.utils.timing:func: execute_query took: 0.038 sec, 0.011 sec CPU time.


                   now
0  2020-08-13 11:54:56


In [35]:
q = """
SELECT
    toDate(event_time) as ds,
    subservice_name as subservice,
    sum(real_consumption_vat) as y
FROM
   cloud_analytics.acquisition_cube
WHERE 1=1
    AND ds >= '2019-10-01'
    AND ds < '2020-08-31'
    AND service_name = 'cloud_ai'
    AND subservice_name in ('speech', 'mt', 'vision')
GROUP BY ds, subservice_name
ORDER BY ds, subservice_name
"""

In [36]:
data = ch.execute_query(q, to_pandas=True)
subservices = data['subservice'].drop_duplicates()

INFO:clan_tools.utils.timing:func: execute_query took: 0.368 sec, 0.013 sec CPU time.


In [37]:
data

,ds,subservice,y
0,2019-10-01,mt,1.628362e+06
1,2019-10-01,speech,3.117068e+05
2,2019-10-01,vision,4.040530e+02
3,2019-10-02,mt,1.274216e+04
4,2019-10-02,speech,3.019615e+05
...,...,...,...
949,2020-08-12,speech,2.582454e+05
950,2020-08-12,vision,1.299275e+03
951,2020-08-13,mt,3.546682e+04
952,2020-08-13,speech,3.994309e+04


In [38]:
subservices

0        mt
1    speech
2    vision
Name: subservice, dtype: object

In [39]:
def get_forecast(data_, days=365, service='unknown'):
    data = data_.copy()
    data.loc[:,'y'] = data.loc[:,'y'].apply(lambda x: x if x<1500000 else np.nan)
    print (data.ds.max())
    m = Prophet(holidays=holidays)
    m.fit(data)
    future = m.make_future_dataframe(periods = days)
    result = m.predict(future)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
#     print(result.iloc[200:250,:])
    result.loc[:,'ds'] = result.loc[:,'ds'].apply(lambda t: str(t)[:10])
    result['service'] = service
    result = result.loc[result['ds'] > data['ds'].max(),:]
#     print (result)
#     print (pd.merge(data, result, on='ds', how='right'))
    return pd.melt(pd.merge(data, result, on='ds', how='right'), id_vars = ['ds', 'service'], value_vars = ['y', 'yhat', 'yhat_lower', 'yhat_upper'])

get_forecast (data.loc[(data['subservice'] == 'mt'),['ds', 'y']], 365, 'mt').head(-10)

2020-08-13


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,service,variable,value
0,2020-08-14,mt,y,NaN
1,2020-08-15,mt,y,NaN
2,2020-08-16,mt,y,NaN
3,2020-08-17,mt,y,NaN
4,2020-08-18,mt,y,NaN
...,...,...,...,...
1445,2021-07-30,mt,yhat_upper,112337.668021
1446,2021-07-31,mt,yhat_upper,90134.632822
1447,2021-08-01,mt,yhat_upper,88036.298049
1448,2021-08-02,mt,yhat_upper,98694.058047


In [40]:
result = []
for subservice in tqdm(['speech','mt','vision']):
    print (subservice)
    result.append(get_forecast(data.loc[(data['subservice'] == subservice),['ds', 'y']], service = subservice + '_actual'))

    result.append(get_forecast(data.loc[(data['subservice'] == subservice),['ds', 'y']].iloc[:-30,:], service = subservice + '_test'))

    data_actual = data.loc[(data['subservice'] == subservice),['ds', 'y']].iloc[-30:,:]
    data_actual['service'] = subservice + '_test'
    data_actual['variable'] = 'y'
    data_actual.rename({'y':'value'}, axis=1, inplace=True)

    result.append(data_actual)

df_res = result[0]

for df_ in result[1:]:
    df_res = df_res.append(df_)

  0%|          | 0/3 [00:00<?, ?it/s]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


speech
2020-08-13


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2020-07-14


 33%|███▎      | 1/3 [00:19<00:39, 19.89s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

 67%|██████▋   | 2/3 [00:39<00:19, 19.89s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


vision
2020-08-13


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2020-07-14


100%|██████████| 3/3 [01:00<00:00, 20.04s/it]


In [41]:
create_table_statement = """

CREATE TABLE cloud_analytics.ml_revenue_forecast(
  ds Date,
  service String,
  variable String,
  value Float64
) ENGINE = MergeTree() ORDER BY(ds, service, variable)

"""

In [42]:
def write_df_to_table(df, table, overwrite=True):
    s = StringIO()
    df.to_csv(s, index=False)
    df_csv = s.getvalue()[s.getvalue().find('\n') + 1:]
    df_cols = s.getvalue()[:s.getvalue().find('\n')]
    df_insert_query = 'INSERT INTO ' +\
                        table +\
                         ' (' +\
                         df_cols +\
                         ') FORMAT CSV ' +\
                         df_csv
    if overwrite:
        try:
            ch.execute_query('DROP TABLE ' + table)
        except:
            pass
        ch.execute_query(create_table_statement)

    ch.execute_query(df_insert_query)

write_df_to_table(df_res[:], 'cloud_analytics.ml_revenue_forecast')


yt.wrapper.config.set_proxy("hahn")

schema = [
            {'name': 'ds', 'type': 'string'},
            {'name': 'service', 'type': 'string'},
            {'name': 'variable', 'type': 'string'},
            {'name': 'value', 'type': 'double'}
        ]
t = '//home/cloud_analytics/ml_metrics/ml_revenue_forecast/ml_revenue_forecast'
try:
    yt.wrapper.create_table(t, attributes={"schema" : schema})
except:
    print('table already exists:', t)


yt.wrapper.write_table(t, df_res.to_dict(orient='records'))

print ('SUCCESS')


INFO:clan_tools.utils.timing:func: execute_query took: 0.055 sec, 0.123 sec CPU time.
INFO:clan_tools.utils.timing:func: execute_query took: 0.099 sec, 0.007 sec CPU time.
INFO:clan_tools.utils.timing:func: execute_query took: 0.049 sec, 0.008 sec CPU time.
/home/artkaz/anaconda3/lib/python3.7/site-packages/yt/common.py:58: YtDeprecationWarning: create_table is deprecated and will be removed in the next major release, use "create" with "table" type instead
  warnings.warn(message, YtDeprecationWarning)


table already exists: //home/cloud_analytics/ml_metrics/ml_revenue_forecast/ml_revenue_forecast
SUCCESS
